# Rate My Professor

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

path = "/usr/local/bin/chromedriver"
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.ratemyprofessors.com/professor/2327680")

wait = WebDriverWait(driver, 10)

course = []
date = []
quality_scores = []
difficulty_score =[]

 # Gets Quality Scores (Different colors have different xpaths)
high_qscores = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[contains(@class, 'CardNumRating__CardNumRatingNumber-sc-17t4b9u-2 gcFhmN')]")))
for score in high_qscores:
    quality_scores.append(score.text)

med_qscores = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[contains(@class, 'CardNumRating__CardNumRatingNumber-sc-17t4b9u-2 icXUyq')]")))
for score in med_qscores:
    quality_scores.append(score.text)

low_qscores = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[contains(@class, 'CardNumRating__CardNumRatingNumber-sc-17t4b9u-2 bUneqk')]")))
for score in low_qscores:
    quality_scores.append(score.text)
    
    # Gets Difficulty Scores
diff_scores = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[contains(@class, 'CardNumRating__CardNumRatingNumber-sc-17t4b9u-2 cDKJcc')]")))
for score in diff_scores:
    difficulty_score.append(score.text)


df = pd.DataFrame({"quality_scores" : quality_scores, "difficulty_scores" : difficulty_score})

df

,quality_scores,difficulty_scores
0,5.0,3.0
1,4.0,4.0
2,4.0,4.0
3,5.0,3.0
4,5.0,3.0
5,4.0,3.0
6,4.0,2.0
7,4.0,3.0
8,3.0,2.0
9,3.0,3.0


In [11]:
from datetime import datetime

# Gets Course Number
course_tag = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[contains(@class, 'RatingHeader__StyledClass-sc-1dlkqw1-3 eXfReS')]")))
for indiv_course in course_tag:
    if indiv_course.text.strip(): # Only true if column doesn't only include whitespace
        course.append(indiv_course.text)

# Gets review date
date_tag = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//div[contains(@class, 'TimeStamp__StyledTimeStamp-sc-9q2r30-0 bXQmMr RatingHeader__RatingTimeStamp-sc-1dlkqw1-4 iwwYJD')]")))
for indiv_date in date_tag:
    if indiv_date.text.strip():
        date.append(indiv_date.text)

driver.quit()

df["course"] = course
df["date"] = date

# Retrieves Year and Semester from df["date"]
def year_and_sem(full_date):
    day_suffix = ["st", "nd", "rd", "th"]
    for suffix in day_suffix:
        if suffix in full_date:
            full_date = full_date.replace(suffix, "")
            break
    date_obj = datetime.strptime(full_date, "%b %d, %Y")
    year = date_obj.year
    month = date_obj.month
    if month == 12 or month == 1:  
        review_semester = "After Fall"
    elif 2 <= month <= 4:  
        review_semester = "During Spring"
    elif 5 <= month <= 9:  
        review_semester = "After Spring"
    else:  
        review_semester = "During Fall"
    return year, review_semester

years = []
review_semesters = []

for date_str in df["date"]:
    year, review_semester = year_and_sem(date_str)
    years.append(year)
    review_semesters.append(review_semester)

df["year"] = years
df["review_semester"] = review_semesters

df

,quality_scores,difficulty_scores,course,date,year,review_semester
0,5.0,3.0,CS124,"Feb 17th, 2025",2025,During Spring
1,4.0,4.0,CS124,"Jan 2nd, 2025",2025,After Fall
2,4.0,4.0,CS124,"Dec 21st, 2024",2024,After Fall
3,5.0,3.0,CS124,"Dec 17th, 2024",2024,After Fall
4,5.0,3.0,CS124,"Dec 15th, 2024",2024,After Fall
5,4.0,3.0,CS124,"Dec 13th, 2024",2024,After Fall
6,4.0,2.0,CS124,"Dec 13th, 2024",2024,After Fall
7,4.0,3.0,CS124,"Dec 10th, 2024",2024,After Fall
8,3.0,2.0,CS124,"Dec 10th, 2024",2024,After Fall
9,3.0,3.0,CS124,"Dec 10th, 2024",2024,After Fall
